<a href="https://colab.research.google.com/github/paruliansaragi/DL-Notebooks/blob/master/allenNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install allennlp

In [0]:
from typing import Iterator, List, Dict
import torch
import torch.optim as optim
import numpy as np
from allennlp.data import Instance
from allennlp.data.fields import TextField, SequenceLabelField
from allennlp.data.dataset_readers import DatasetReader
from allennlp.common.file_utils import cached_path
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.data.vocabulary import Vocabulary
from allennlp.models import Model
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.seq2seq_encoders import Seq2SeqEncoder, PytorchSeq2SeqWrapper
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits
from allennlp.training.metrics import CategoricalAccuracy
from allennlp.data.iterators import BucketIterator
from allennlp.training.trainer import Trainer
from allennlp.predictors import SentenceTaggerPredictor

torch.manual_seed(1)

In [0]:
class PosDatasetReader(DatasetReader):
    """
    DatasetReader for PoS tagging data, one sentence per line, like

        The###DET dog###NN ate###V the###DET apple###NN
    """
    def __init__(self, token_indexers: Dict[str, TokenIndexer] = None) -> None:
        super().__init__(lazy=False)
        self.token_indexers = token_indexers or {"tokens": SingleIdTokenIndexer()}
    def text_to_instance(self, tokens: List[Token], tags: List[str] = None) -> Instance:
        sentence_field = TextField(tokens, self.token_indexers)
        fields = {"sentence": sentence_field}

        if tags:
            label_field = SequenceLabelField(labels=tags, sequence_field=sentence_field)
            fields["labels"] = label_field

        return Instance(fields)
    def _read(self, file_path: str) -> Iterator[Instance]:
        with open(file_path) as f:
            for line in f:
                pairs = line.strip().split()
                sentence, tags = zip(*(pair.split("###") for pair in pairs))
                yield self.text_to_instance([Token(word) for word in sentence], tags)

In [0]:
class LstmTagger(Model):
    def __init__(self,
                 word_embeddings: TextFieldEmbedder,#insert BERT/ELMO here
                 encoder: Seq2SeqEncoder,
                 vocab: Vocabulary) -> None:
        super().__init__(vocab)
        self.word_embeddings = word_embeddings
        self.encoder = encoder
        self.hidden2tag = torch.nn.Linear(in_features=encoder.get_output_dim(),
                                          out_features=vocab.get_vocab_size('labels'))
        self.accuracy = CategoricalAccuracy()
        
    def forward(self,
                sentence: Dict[str, torch.Tensor],
                labels: torch.Tensor = None) -> Dict[str, torch.Tensor]:
        mask = get_text_field_mask(sentence)
        embeddings = self.word_embeddings(sentence)
        encoder_out = self.encoder(embeddings, mask)
        tag_logits = self.hidden2tag(encoder_out)
        output = {"tag_logits": tag_logits}
        if labels is not None:
            self.accuracy(tag_logits, labels, mask)
            output["loss"] = sequence_cross_entropy_with_logits(tag_logits, labels, mask)

        return output
    def get_metrics(self, reset: bool = False) -> Dict[str, float]:
        return {"accuracy": self.accuracy.get_metric(reset)}

In [0]:
reader = PosDatasetReader()
train_dataset = reader.read(cached_path(
    'https://raw.githubusercontent.com/allenai/allennlp'
    '/master/tutorials/tagger/training.txt'))

validation_dataset = reader.read(cached_path(
    'https://raw.githubusercontent.com/allenai/allennlp'
    '/master/tutorials/tagger/validation.txt'))

vocab = Vocabulary.from_instances(train_dataset + validation_dataset)

02/15/2019 12:57:45 - INFO - allennlp.common.file_utils -   https://raw.githubusercontent.com/allenai/allennlp/master/tutorials/tagger/training.txt not found in cache, downloading to /tmp/tmptum6vf7e
93B [00:00, 20691.19B/s]             
02/15/2019 12:57:45 - INFO - allennlp.common.file_utils -   copying /tmp/tmptum6vf7e to cache at /root/.allennlp/cache/c3e1f451545a79cf7582dec24d072db6f5bb0d1ae24a924d03c9944516e16b60.47b1193282cbd926a1b602cc6d5a22324cfab24e669ca04f1ff4851a35c73393
02/15/2019 12:57:45 - INFO - allennlp.common.file_utils -   creating metadata file for /root/.allennlp/cache/c3e1f451545a79cf7582dec24d072db6f5bb0d1ae24a924d03c9944516e16b60.47b1193282cbd926a1b602cc6d5a22324cfab24e669ca04f1ff4851a35c73393
02/15/2019 12:57:45 - INFO - allennlp.common.file_utils -   removing temp file /tmp/tmptum6vf7e
2it [00:00, 4401.16it/s]
02/15/2019 12:57:45 - INFO - allennlp.common.file_utils -   https://raw.githubusercontent.com/allenai/allennlp/master/tutorials/tagger/validation.txt not

In [0]:
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                            embedding_dim=EMBEDDING_DIM)

word_embeddings = BasicTextFieldEmbedder({"tokens": token_embedding})

In [0]:
lstm = PytorchSeq2SeqWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))

model = LstmTagger(word_embeddings, lstm, vocab)

optimizer = optim.SGD(model.parameters(), lr=0.1)

iterator = BucketIterator(batch_size=2, sorting_keys=[("sentence", "num_tokens")])

iterator.index_with(vocab)

In [0]:
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=validation_dataset,
                  patience=10,
                  num_epochs=1000)

trainer.train()

02/15/2019 12:57:53 - INFO - allennlp.training.trainer -   Beginning training.
02/15/2019 12:57:53 - INFO - allennlp.training.trainer -   Epoch 0/999
02/15/2019 12:57:53 - INFO - allennlp.training.trainer -   Peak CPU memory usage MB: 375.148
02/15/2019 12:57:54 - INFO - allennlp.training.trainer -   GPU 0 memory usage MB: 11
02/15/2019 12:57:54 - INFO - allennlp.training.trainer -   Training
accuracy: 0.3333, loss: 1.1685 ||: 100%|██████████| 1/1 [00:00<00:00,  7.15it/s]
02/15/2019 12:57:54 - INFO - allennlp.training.trainer -   Validating
accuracy: 0.3333, loss: 1.1592 ||: 100%|██████████| 1/1 [00:00<00:00, 312.38it/s]
02/15/2019 12:57:54 - INFO - allennlp.training.trainer -                       Training |  Validation
02/15/2019 12:57:54 - INFO - allennlp.training.trainer -   cpu_memory_MB   |   375.148  |       N/A
02/15/2019 12:57:54 - INFO - allennlp.training.trainer -   loss            |     1.168  |     1.159
02/15/2019 12:57:54 - INFO - allennlp.training.trainer -   accuracy  

{'best_epoch': 999,
 'best_validation_accuracy': 1.0,
 'best_validation_loss': 0.01804003119468689,
 'epoch': 999,
 'peak_cpu_memory_MB': 384.908,
 'peak_gpu_0_memory_MB': 11,
 'training_accuracy': 1.0,
 'training_cpu_memory_MB': 384.908,
 'training_duration': '00:01:12',
 'training_epochs': 999,
 'training_gpu_0_memory_MB': 11,
 'training_loss': 0.01809469424188137,
 'training_start_epoch': 0,
 'validation_accuracy': 1.0,
 'validation_loss': 0.01804003119468689}

In [0]:
predictor = SentenceTaggerPredictor(model, dataset_reader=reader)

tag_logits = predictor.predict("The dog ate the apple")['tag_logits']

In [0]:
tag_ids = np.argmax(tag_logits, axis=-1)

print([model.vocab.get_token_from_index(i, 'labels') for i in tag_ids])

['DET', 'NN', 'V', 'DET', 'NN']


In [0]:
# Here's how to save the model.
with open("/tmp/model.th", 'wb') as f:
    torch.save(model.state_dict(), f)
vocab.save_to_files("/tmp/vocabulary")

In [0]:
# And here's how to reload the model.
vocab2 = Vocabulary.from_files("/tmp/vocabulary")
model2 = LstmTagger(word_embeddings, lstm, vocab2)
with open("/tmp/model.th", 'rb') as f:
    model2.load_state_dict(torch.load(f))

02/15/2019 13:01:47 - INFO - allennlp.data.vocabulary -   Loading token dictionary from /tmp/vocabulary.


In [0]:
predictor2 = SentenceTaggerPredictor(model2, dataset_reader=reader)
tag_logits2 = predictor2.predict("The dog ate the apple")['tag_logits']
assert tag_logits2 == tag_logits

In [0]:
tag_ids2 = np.argmax(tag_logits2, axis=-1)

print([model2.vocab.get_token_from_index(i, 'labels') for i in tag_ids2])

['DET', 'NN', 'V', 'DET', 'NN']


https://github.com/TheShadow29/ALNLP-Notes/blob/master/notebooks/ALNLP-Tutorial.ipynb

https://github.com/TheShadow29/ALNLP-Notes/blob/master/notebooks/Using-Bert.ipynb